In [1]:
import os
from crewai import Agent, Task, Crew
from langchain.llms import OpenAI
# from langchain.llms import Ollama

##### In order to use Crew.ai with Ollama, we should set OpenAI's environment variables with these values:

##### export OPENAI_API_BASE='http://localhost:11434/v1'
##### export OPENAI_MODEL_NAME='mistral'
##### export OPENAI_API_KEY=NA

In [ ]:
# llm = Ollama(
#     model="mistral",
#     base_url="http://localhost:11434"
# )

# Defining the agents
product_manager = Agent(
    role="Product Manager",
    goal="Oversee the project and ensure the game meets user needs and business goals",
    backstory="""
        You are an experienced Product Manager at the famous "Golan Heights Games Company" with a 
        knack for understanding user needs and translating them into product requirements.
        You have successfully launched several software products in the past and are known 
        for your strategic thinking and leadership skills.
    """,
    allow_delegation=False,
    verbose=True
)

software_engineer = Agent(
    role="Software Engineer",
    goal="Building quality software",
    backstory="""
        You are a talented Software Engineer at the famous "Golan Heights Games Company" with a passion 
        for coding and problem-solving.
        You enjoy working on challenging projects and have a strong background in game development.
        For the last 6 years you have been developing games using Python.
        You deliver fast and quality code and you are always open to a constructive criticism.
    """,
    allow_delegation=False,
    verbose=True
)

senior_software_engineer = Agent(
    role="Senior Software Engineer",
    goal="Lead the development team and ensure code quality and best practices",
    backstory="""
        You are a seasoned Senior Software Engineer at the famous "Golan Heights Games Company" with extensive 
        experience in software architecture and team leadership.
        You have a deep understanding of coding standards and best practices and are committed to mentoring junior developers.
        You are also a Python expert.
        You are commited to the company and to the quality of the product.
        You are kind but asertive and you will not hesitate to ask your workers to refactor
        their code when you believe its needed.
    """,
    allow_delegation=True,
    verbose=True
)

qa_engineer = Agent(
    role="QA Engineer",
    goal="Test the game to ensure it is free of bugs and provides a smooth user experience",
    backstory="""
        You are a meticulous QA Engineer at the famous "Golan Heights Games Company" with a sharp 
        eye for detail.
        You excel at finding and documenting bugs and have a strong commitment 
        to delivering high-quality software products.
    """,
    allow_delegation=True,
    verbose=True
)

# Defining the tasks
task1 = Task(
    description="Define the project scope and requirements for the Tic Tac Toe game.",
    expected_output="A detailed project scope document outlining game features, user stories, and acceptance criteria.",
    agent=product_manager
)

task2 = Task(
    description="Design and implement the game logic for Tic Tac Toe in Python",
    expected_output="The full Well-documented code implementing the Tic Tac Toe game logic.",
    agent=software_engineer,
    dependencies=[task1]
)

task3 = Task(
    description="Review and optimize the game code to ensure high performance and maintainability.",
    expected_output="The full Optimized and well-structured game code with comments explaining key sections.",
    agent=senior_software_engineer,
    dependencies=[task2]
)

task4 = Task(
    description="""
        Test the game thoroughly to identify and document any bugs or issues that should be 
        fixed before the game is ready for deployment.
        Your Final answer must be the full python code, only the python code and nothing else.
    """,
    expected_output="The full final validated python code to run the game.",
    agent=qa_engineer,
    dependencies=[task3],
)

# Creating the crew
crew = Crew(
    agents=[product_manager, software_engineer, senior_software_engineer, qa_engineer],
    tasks=[task1, task2, task3, task4],
    verbose=2
)

# Kicking off the crew to start the project
result = crew.kickoff()

print("############")
print(result)


In [3]:
# Import Pygame library
import pygame
import random

# Initialize Pygame
pygame.init()

# Set screen dimensions and create a screen object
screen_width = 800
screen_height = 600
screen = pygame.display.set_mode((screen_width, screen_height))

# Set title and clock
pygame.display.set_caption("Simple Game")
clock = pygame.time.Clock()

# Define colors
WHITE = (255, 255, 255)
BLACK = (0, 0, 0)
RED = (255, 0, 0)
GREEN = (0, 255, 0)

# Player properties
player_size = 50
player_pos = [screen_width//2, screen_height-2*player_size]
player_speed = 10

# Enemy properties
enemy_size = 50
enemy_pos = [random.randint(0, screen_width-enemy_size), 0]
enemy_speed = 5

# Function to detect collisions
def detect_collision(player_pos, enemy_pos):
    p_x, p_y = player_pos
    e_x, e_y = enemy_pos
    
    if (e_x >= p_x and e_x < (p_x + player_size)) or (p_x >= e_x and p_x < (e_x + enemy_size)):
        if (e_y >= p_y and e_y < (p_y + player_size)) or (p_y >= e_y and p_y < (e_y + enemy_size)):
            return True
    return False

# Game loop
game_over = False
while not game_over:
    for event in pygame.event.get():
        if event.type == pygame.QUIT:
            game_over = True

    keys = pygame.key.get_pressed()
    
    if keys[pygame.K_LEFT]:
        player_pos[0] -= player_speed
    if keys[pygame.K_RIGHT]:
        player_pos[0] += player_speed

    screen.fill(BLACK)

    # Update enemy position
    enemy_pos[1] += enemy_speed
    if enemy_pos[1] > screen_height:
        enemy_pos[0] = random.randint(0, screen_width-enemy_size)
        enemy_pos[1] = 0

    # Check for collision
    if detect_collision(player_pos, enemy_pos):
        game_over = True
        break

    # Draw player and enemy
    pygame.draw.rect(screen, GREEN, (player_pos[0], player_pos[1], player_size, player_size))
    pygame.draw.rect(screen, RED, (enemy_pos[0], enemy_pos[1], enemy_size, enemy_size))

    pygame.display.update()

    clock.tick(30)

# Quit Pygame
pygame.quit()

pygame 2.6.0 (SDL 2.28.4, Python 3.12.3)
Hello from the pygame community. https://www.pygame.org/contribute.html


In [1]:
class TicTacToe:
    def __init__(self):
        self.board = [' ' for _ in range(9)]
        self.current_winner = None  # keep track of the winner!

    def print_board(self):
        # this is just getting the rows
        for row in [self.board[i * 3:(i + 1) * 3] for i in range(3)]:
            print('| ' + ' | '.join(row) + ' |')

    @staticmethod
    def print_board_nums():
        # 0 | 1 | 2 etc (tells us what number corresponds to what box)
        number_board = [[str(i) for i in range(j * 3, (j + 1) * 3)] for j in range(3)]
        for row in number_board:
            print('| ' + ' | '.join(row) + ' |')

    def available_moves(self):
        return [i for i, spot in enumerate(self.board) if spot == ' ']

    def empty_squares(self):
        return ' ' in self.board

    def num_empty_squares(self):
        return self.board.count(' ')

    def make_move(self, square, letter):
        # if valid move, then make the move (assign square to letter)
        # then return true. if invalid, return false
        if self.board[square] == ' ':
            self.board[square] = letter
            if self.winner(square, letter):
                self.current_winner = letter
            return True
        return False

    def winner(self, square, letter):
        # winner if 3 in a row anywhere.. we have to check all of these!
        # first let's check the row
        row_ind = square // 3
        row = self.board[row_ind * 3:(row_ind + 1) * 3]
        if all([spot == letter for spot in row]):
            return True
        # check column
        col_ind = square % 3
        column = [self.board[col_ind + i * 3] for i in range(3)]
        if all([spot == letter for spot in column]):
            return True
        # check diagonals
        # these are the only two diagonals
        if square % 2 == 0:
            diagonal1 = [self.board[i] for i in [0, 4, 8]]
            if all([spot == letter for spot in diagonal1]):
                return True
            diagonal2 = [self.board[i] for i in [2, 4, 6]]
            if all([spot == letter for spot in diagonal2]):
                return True
        # if all of these fail
        return False


def play(game, x_player, o_player, print_game=True):
    if print_game:
        game.print_board_nums()

    letter = 'X'  # starting letter
    # iterate while the game still has empty squares
    while game.empty_squares():
        if game.num_empty_squares() == 9:
            # make the move
            square = x_player.get_move(game)
        else:
            square = o_player.get_move(game)
        # let's define a function to make a move!
        if game.make_move(square, letter):
            if print_game:
                print(letter + f' makes a move to square {square}')
                game.print_board()
                print('')  # just empty line

            if game.current_winner:
                if print_game:
                    print(letter + ' wins!')
                return letter  # ends the loop and exits the game
            # after we made our move, we need to alternate letters
            letter = 'O' if letter == 'X' else 'X'  # switches player

        # tiny break to make things a little easier to read
    if print_game:
        print('It\'s a tie!')


class HumanPlayer:
    def __init__(self, letter):
        self.letter = letter

    def get_move(self, game):
        valid_square = False
        val = None
        while not valid_square:
            square = input(self.letter + '\'s turn. Input move (0-8): ')
            try:
                val = int(square)
                if val not in game.available_moves():
                    raise ValueError
                valid_square = True
            except ValueError:
                print('Invalid square. Try again.')
        return val


class RandomComputerPlayer:
    def __init__(self, letter):
        self.letter = letter

    def get_move(self, game):
        import random
        square = random.choice(game.available_moves())
        return square


class GeniusComputerPlayer:
    def __init__(self, letter):
        self.letter = letter

    def get_move(self, game):
        if len(game.available_moves()) == 9:
            square = 4
        else:
            square = self.minimax(game, self.letter)['position']
        return square

    def minimax(self, state, player):
        max_player = self.letter  # yourself!!
        other_player = 'O' if player == 'X' else 'X'

        if state.current_winner == other_player:
            return {'position': None, 'score': 1 * (state.num_empty_squares() + 1) if other_player == max_player else -1 * (state.num_empty_squares() + 1)}

        elif not state.empty_squares():
            return {'position': None, 'score': 0}

        if player == max_player:
            best = {'position': None, 'score': -float('inf')}
        else:
            best = {'position': None, 'score': float('inf')}
        for possible_move in state.available_moves():
            state.make_move(possible_move, player)
            sim_score = self.minimax(state, other_player)
            state.board[possible_move] = ' '
            state.current_winner = None
            sim_score['position'] = possible_move

            if player == max_player:
                if sim_score['score'] > best['score']:
                    best = sim_score
            else:
                if sim_score['score'] < best['score']:
                    best = sim_score
        return best


if __name__ == '__main__':
    x_player = HumanPlayer('X')
    o_player = GeniusComputerPlayer('O')
    t = TicTacToe()
    play(t, x_player, o_player, print_game=True)

| 0 | 1 | 2 |
| 3 | 4 | 5 |
| 6 | 7 | 8 |


X's turn. Input move (0-8):  0


X makes a move to square 0
| X |   |   |
|   |   |   |
|   |   |   |

O makes a move to square 4
| X |   |   |
|   | O |   |
|   |   |   |

X makes a move to square 1
| X | X |   |
|   | O |   |
|   |   |   |

O makes a move to square 2
| X | X | O |
|   | O |   |
|   |   |   |

X makes a move to square 6
| X | X | O |
|   | O |   |
| X |   |   |

O makes a move to square 3
| X | X | O |
| O | O |   |
| X |   |   |

X makes a move to square 5
| X | X | O |
| O | O | X |
| X |   |   |

O makes a move to square 7
| X | X | O |
| O | O | X |
| X | O |   |

X makes a move to square 8
| X | X | O |
| O | O | X |
| X | O | X |

It's a tie!


In [2]:

class TicTacToe:
    def __init__(self):
        """Initialize the Tic Tac Toe board and set the current winner to None."""
        self.board = [' ' for _ in range(9)]
        self.current_winner = None

    def print_board(self):
        """Print the current game board."""
        for row in [self.board[i * 3:(i + 1) * 3] for i in range(3)]:
            print('| ' + ' | '.join(row) + ' |')

    @staticmethod
    def print_board_nums():
        """Print the board numbers for reference."""
        number_board = [[str(i) for i in range(j * 3, (j + 1) * 3)] for j in range(3)]
        for row in number_board:
            print('| ' + ' | '.join(row) + ' |')

    def available_moves(self):
        """Return a list of available moves (indices) on the board."""
        return [i for i, spot in enumerate(self.board) if spot == ' ']

    def empty_squares(self):
        """Check if there are any empty squares on the board."""
        return ' ' in self.board

    def num_empty_squares(self):
        """Return the number of empty squares on the board."""
        return self.board.count(' ')

    def make_move(self, square, letter):
        """
        Make a move on the board.
        
        Args:
        square (int): The square to place the letter.
        letter (str): The player's letter ('X' or 'O').
        
        Returns:
        bool: True if the move is valid, False otherwise.
        """
        if self.board[square] == ' ':
            self.board[square] = letter
            if self.winner(square, letter):
                self.current_winner = letter
            return True
        return False

    def winner(self, square, letter):
        """
        Check if the given move results in a win.
        
        Args:
        square (int): The square to check.
        letter (str): The player's letter ('X' or 'O').
        
        Returns:
        bool: True if the move results in a win, False otherwise.
        """
        # Check the row
        row_ind = square // 3
        row = self.board[row_ind * 3:(row_ind + 1) * 3]
        if all([spot == letter for spot in row]):
            return True

        # Check the column
        col_ind = square % 3
        column = [self.board[col_ind + i * 3] for i in range(3)]
        if all([spot == letter for spot in column]):
            return True

        # Check diagonals
        if square % 2 == 0:
            diagonal1 = [self.board[i] for i in [0, 4, 8]]
            if all([spot == letter for spot in diagonal1]):
                return True
            diagonal2 = [self.board[i] for i in [2, 4, 6]]
            if all([spot == letter for spot in diagonal2]):
                return True

        return False


def play(game, x_player, o_player, print_game=True):
    """
    Play a game of Tic Tac Toe.
    
    Args:
    game (TicTacToe): The game instance.
    x_player: The player object for 'X'.
    o_player: The player object for 'O'.
    print_game (bool): Whether to print the game board during play.
    
    Returns:
    str: The letter of the winning player, or None if it's a tie.
    """
    if print_game:
        game.print_board_nums()

    letter = 'X'  # Starting letter
    while game.empty_squares():
        if game.num_empty_squares() == 9:
            square = x_player.get_move(game)
        else:
            square = o_player.get_move(game)

        if game.make_move(square, letter):
            if print_game:
                print(letter + f' makes a move to square {square}')
                game.print_board()
                print('')

            if game.current_winner:
                if print_game:
                    print(letter + ' wins!')
                return letter

            letter = 'O' if letter == 'X' else 'X'

    if print_game:
        print('It\'s a tie!')


class HumanPlayer:
    def __init__(self, letter):
        """Initialize the human player with a letter ('X' or 'O')."""
        self.letter = letter

    def get_move(self, game):
        """
        Get the move from the human player.
        
        Args:
        game (TicTacToe): The game instance.
        
        Returns:
        int: The square chosen by the player.
        """
        valid_square = False
        val = None
        while not valid_square:
            square = input(self.letter + '\'s turn. Input move (0-8): ')
            try:
                val = int(square)
                if val not in game.available_moves():
                    raise ValueError
                valid_square = True
            except ValueError:
                print('Invalid square. Try again.')
        return val


class RandomComputerPlayer:
    def __init__(self, letter):
        """Initialize the computer player with a letter ('X' or 'O')."""
        self.letter = letter

    def get_move(self, game):
        """
        Get a random move from the computer player.
        
        Args:
        game (TicTacToe): The game instance.
        
        Returns:
        int: The square chosen by the computer.
        """
        import random
        square = random.choice(game.available_moves())
        return square


class GeniusComputerPlayer:
    def __init__(self, letter):
        """Initialize the genius computer player with a letter ('X' or 'O')."""
        self.letter = letter

    def get_move(self, game):
        """
        Get the best move from the genius computer player using the minimax algorithm.
        
        Args:
        game (TicTacToe): The game instance.
        
        Returns:
        int: The square chosen by the computer.
        """
        if len(game.available_moves()) == 9:
            square = 4
        else:
            square = self.minimax(game, self.letter)['position']
        return square

    def minimax(self, state, player):
        """
        Minimax algorithm to determine the best move.
        
        Args:
        state (TicTacToe): The current game state.
        player (str): The player's letter ('X' or 'O').
        
        Returns:
        dict: The best position and score.
        """
        max_player = self.letter
        other_player = 'O' if player == 'X' else 'X'

        if state.current_winner == other_player:
            return {'position': None, 'score': 1 * (state.num_empty_squares() + 1) if other_player == max_player else -1 * (state.num_empty_squares() + 1)}

        elif not state.empty_squares():
            return {'position': None, 'score': 0}

        if player == max_player:
            best = {'position': None, 'score': -float('inf')}
        else:
            best = {'position': None, 'score': float('inf')}
        
        for possible_move in state.available_moves():
            state.make_move(possible_move, player)
            sim_score = self.minimax(state, other_player)
            state.board[possible_move] = ' '
            state.current_winner = None
            sim_score['position'] = possible_move

            if player == max_player:
                if sim_score['score'] > best['score']:
                    best = sim_score
            else:
                if sim_score['score'] < best['score']:
                    best = sim_score
        return best


if __name__ == '__main__':
    x_player = HumanPlayer('X')
    o_player = GeniusComputerPlayer('O')
    t = TicTacToe()
    play(t, x_player, o_player, print_game=True)

| 0 | 1 | 2 |
| 3 | 4 | 5 |
| 6 | 7 | 8 |


X's turn. Input move (0-8):  0


X makes a move to square 0
| X |   |   |
|   |   |   |
|   |   |   |

O makes a move to square 4
| X |   |   |
|   | O |   |
|   |   |   |

X makes a move to square 1
| X | X |   |
|   | O |   |
|   |   |   |

O makes a move to square 2
| X | X | O |
|   | O |   |
|   |   |   |

X makes a move to square 6
| X | X | O |
|   | O |   |
| X |   |   |

O makes a move to square 3
| X | X | O |
| O | O |   |
| X |   |   |

X makes a move to square 5
| X | X | O |
| O | O | X |
| X |   |   |

O makes a move to square 7
| X | X | O |
| O | O | X |
| X | O |   |

X makes a move to square 8
| X | X | O |
| O | O | X |
| X | O | X |

It's a tie!


In [1]:
class TicTacToe:
    def __init__(self):
        self.board = [[' ' for _ in range(3)] for _ in range(3)]
        self.current_turn = 'X'
        self.winner = None
        self.moves_made = []

    def print_board(self):
        for row in self.board:
            print('|'.join(row))
            print('-' * 5)

    def make_move(self, row, col):
        if self.board[row][col] == ' ' and self.winner is None:
            self.board[row][col] = self.current_turn
            self.moves_made.append((row, col))
            if self.check_winner(row, col):
                self.winner = self.current_turn
            elif self.is_full():
                self.winner = 'Tie'
            else:
                self.current_turn = 'O' if self.current_turn == 'X' else 'X'
            return True
        return False

    def undo_move(self):
        if self.moves_made:
            last_move = self.moves_made.pop()
            self.board[last_move[0]][last_move[1]] = ' '
            self.current_turn = 'O' if self.current_turn == 'X' else 'X'
            self.winner = None
            # Recalculate the winner if necessary
            if self.moves_made:
                last_move = self.moves_made[-1]
                self.check_winner(last_move[0], last_move[1])

    def check_winner(self, row, col):
        if self.check_rows(row) or self.check_columns(col) or self.check_diagonals():
            self.winner = self.current_turn
            return True
        return False

    def check_rows(self, row):
        return all(self.board[row][i] == self.current_turn for i in range(3))

    def check_columns(self, col):
        return all(self.board[i][col] == self.current_turn for i in range(3))

    def check_diagonals(self):
        return (all(self.board[i][i] == self.current_turn for i in range(3)) or
                all(self.board[i][2 - i] == self.current_turn for i in range(3)))

    def is_full(self):
        return all(self.board[row][col] != ' ' for row in range(3) for col in range(3))

    def get_winner(self):
        return self.winner

    def reset_game(self):
        self.__init__()

def get_player_input(player):
    while True:
        try:
            row = int(input(f"Player {player}, enter row (0, 1, 2): "))
            col = int(input(f"Player {player}, enter column (0, 1, 2): "))
            if row in range(3) and col in range(3):
                return row, col
            else:
                print("Invalid input. Please enter numbers between 0 and 2.")
        except ValueError:
            print("Invalid input. Please enter valid numbers.")

def play_game():
    game = TicTacToe()
    while not game.is_full() and game.get_winner() is None:
        game.print_board()
        row, col = get_player_input(game.current_turn)
        if not game.make_move(row, col):
            print("Invalid move, try again.")

    game.print_board()
    if game.get_winner() and game.get_winner() != 'Tie':
        print(f"Player {game.get_winner()} wins!")
    else:
        print("It's a tie!")

if __name__ == "__main__":
    play_game()

 | | 
-----
 | | 
-----
 | | 
-----


Player X, enter row (0, 1, 2):  0
Player X, enter column (0, 1, 2):  0


X| | 
-----
 | | 
-----
 | | 
-----


Player O, enter row (0, 1, 2):  1
Player O, enter column (0, 1, 2):  1


X| | 
-----
 |O| 
-----
 | | 
-----


...

X| |X
-----
 |O| 
-----
O| |X
-----


Player O, enter row (0, 1, 2):  0
Player O, enter column (0, 1, 2):  1


X|O|X
-----
 |O| 
-----
O| |X
-----


Player X, enter row (0, 1, 2):  1
Player X, enter column (0, 1, 2):  2


X|O|X
-----
 |O|X
-----
O| |X
-----
Player X wins!


In [ ]:
# Import the necessary PyGame module
import pygame
import sys

# Initialize the PyGame library
pygame.init()

# Set up the display variables
WIDTH, HEIGHT = 300, 300
LINE_WIDTH = 15
BOARD_ROWS = 3
BOARD_COLS = 3
SQUARE_SIZE = WIDTH // BOARD_COLS
CIRCLE_RADIUS = SQUARE_SIZE // 3
CIRCLE_WIDTH = 15
CROSS_WIDTH = 25
SPACE = SQUARE_SIZE // 4

# Color variables (R, G, B)
BG_COLOR = (28, 170, 156)
LINE_COLOR = (23, 145, 135)
CIRCLE_COLOR = (239, 231, 200)
CROSS_COLOR = (66, 66, 66)

# Set up the display
screen = pygame.display.set_mode((WIDTH, HEIGHT))
pygame.display.set_caption('Tic Tac Toe')
screen.fill(BG_COLOR)

# Create the board
board = [[0 for _ in range(BOARD_COLS)] for _ in range(BOARD_ROWS)]

# Draw the game grid
def draw_lines():
    """Draws the grid lines for the Tic Tac Toe board."""
    for row in range(1, BOARD_ROWS):
        pygame.draw.line(screen, LINE_COLOR, (0, row * SQUARE_SIZE), 
                         (WIDTH, row * SQUARE_SIZE), LINE_WIDTH)
        pygame.draw.line(screen, LINE_COLOR, (row * SQUARE_SIZE, 0), 
                         (row * SQUARE_SIZE, HEIGHT), LINE_WIDTH)

# Draw a figure (circle or cross)
def draw_figure(player, row, col):
    """Draws a circle or a cross based on the player value."""
    center = (int(col * SQUARE_SIZE + SQUARE_SIZE // 2), 
              int(row * SQUARE_SIZE + SQUARE_SIZE // 2))
    if player == 1:
        pygame.draw.circle(screen, CIRCLE_COLOR, center, CIRCLE_RADIUS, CIRCLE_WIDTH)
    elif player == 2:
        pygame.draw.line(screen, CROSS_COLOR, 
                         (col * SQUARE_SIZE + SPACE, row * SQUARE_SIZE + SQUARE_SIZE - SPACE), 
                         (col * SQUARE_SIZE + SQUARE_SIZE - SPACE, row * SQUARE_SIZE + SPACE), 
                         CROSS_WIDTH)
        pygame.draw.line(screen, CROSS_COLOR, 
                         (col * SQUARE_SIZE + SPACE, row * SQUARE_SIZE + SPACE), 
                         (col * SQUARE_SIZE + SQUARE_SIZE - SPACE, row * SQUARE_SIZE + SQUARE_SIZE - SPACE), 
                         CROSS_WIDTH)

# Draw all figures on the board
def draw_figures():
    """Draws all the figures (circles and crosses) on the board."""
    for row in range(BOARD_ROWS):
        for col in range(BOARD_COLS):
            if board[row][col] != 0:
                draw_figure(board[row][col], row, col)

# Mark the square on board
def mark_square(row, col, player):
    """Marks the board cell with the player's value."""
    board[row][col] = player

# Check if the square is available
def available_square(row, col):
    """Checks if a board cell is available for marking."""
    return board[row][col] == 0

# Check if the board is full
def is_board_full():
    """Checks if the board is completely filled."""
    for row in range(BOARD_ROWS):
        for col in range(BOARD_COLS):
            if board[row][col] == 0:
                return False
    return True

# Check for win
def check_win(player):
    """Checks if the specified player has won the game."""
    # Vertical win check
    for col in range(BOARD_COLS):
        if all(board[row][col] == player for row in range(BOARD_ROWS)):
            draw_winning_line('vertical', col, player)
            return True

    # Horizontal win check
    for row in range(BOARD_ROWS):
        if all(board[row][col] == player for col in range(BOARD_COLS)):
            draw_winning_line('horizontal', row, player)
            return True

    # Ascending diagonal win check
    if all(board[i][BOARD_COLS - 1 - i] == player for i in range(BOARD_ROWS)):
        draw_winning_line('asc_diagonal', None, player)
        return True

    # Descending diagonal win check
    if all(board[i][i] == player for i in range(BOARD_ROWS)):
        draw_winning_line('desc_diagonal', None, player)
        return True

    return False

# Draw winning line
def draw_winning_line(direction, index, player):
    """Draws the winning line based on the direction and player."""
    color = CIRCLE_COLOR if player == 1 else CROSS_COLOR
    if direction == 'vertical':
        posX = index * SQUARE_SIZE + SQUARE_SIZE // 2
        pygame.draw.line(screen, color, (posX, 15), (posX, HEIGHT - 15), LINE_WIDTH)
    elif direction == 'horizontal':
        posY = index * SQUARE_SIZE + SQUARE_SIZE // 2
        pygame.draw.line(screen, color, (15, posY), (WIDTH - 15, posY), LINE_WIDTH)
    elif direction == 'asc_diagonal':
        pygame.draw.line(screen, color, (15, HEIGHT - 15), (WIDTH - 15, 15), LINE_WIDTH)
    elif direction == 'desc_diagonal':
        pygame.draw.line(screen, color, (15, 15), (WIDTH - 15, HEIGHT - 15), LINE_WIDTH)

# Restart the game
def restart():
    """Restarts the game by resetting the board and drawing the lines."""
    screen.fill(BG_COLOR)
    draw_lines()
    for row in range(BOARD_ROWS):
        for col in range(BOARD_COLS):
            board[row][col] = 0

# Main game loop
draw_lines()
player = 1
game_over = False

while True:
    for event in pygame.event.get():
        if event.type == pygame.QUIT:
            pygame.quit()
            sys.exit()
        if event.type == pygame.MOUSEBUTTONDOWN and not game_over:
            mouseX = event.pos[0]  # X coordinate of the click
            mouseY = event.pos[1]  # Y coordinate of the click

            clicked_row = mouseY // SQUARE_SIZE
            clicked_col = mouseX // SQUARE_SIZE

            if available_square(clicked_row, clicked_col):
                mark_square(clicked_row, clicked_col, player)
                if check_win(player):
                    game_over = True
                player = 3 - player  # Switch between player 1 and 2
                draw_figures()

        if event.type == pygame.KEYDOWN:
            if event.key == pygame.K_r:
                restart()
                game_over = False

    pygame.display.update()

pygame 2.6.0 (SDL 2.28.4, Python 3.12.3)
Hello from the pygame community. https://www.pygame.org/contribute.html
